# Estimation of correlation between centroid offset and $\Sigma_{10, \rm flux\ selected}$

**Author(s):** Muhammad Jobair Hasan, Anowar J. Shajib

## Uncertainty propagation and the Pearson correlation coeffient (PCC)

We assume that the centroids (`"center_x_light"`, `"center_x_mass"`, `"center_y_light"`, `"center_y_mass"`) conform to Gaussian distributions of means equaling the corresponding medians and the standard deviations equaling the corresponding averages (of the upper and the lower) $1\sigma$ uncertainties. These uncertainties propagate (in our calculations using the Euclidean distance formula) to the values of light and mass centeroid offsets. We then use normal distributions having the calculated (uncertainties propagated) means and uncertainties (standard deviations) to sample from and calculate the Pearson correlation cofficient (PCC) between the centeroid offsets and the $\Sigma_{10, \rm flux\ selected}$ values. By sampling multiple times and calculating the PCCs we get a population of PCC values and thus calculate the mean, $1\sigma$ upper and $1\sigma$ lower uncertainties.

## Relevant formulae


For the case of $f = aA + bB$, where A and B are two random variables and a, b are constants, we have $$\sigma_{f} = \sqrt{a^{2}\sigma_{A}^2 + b^{2}\sigma_{B}^2 - 2ab\sigma_{AB}}$$
In the case of independent A, B ($\sigma_{AB}=0$) and $a=b=1$ we have $$\sigma_{f} = \sqrt{\sigma_{A}^2 + \sigma_{B}^2}$$
We also have for independent A, B ($\sigma_{AB}=0$) and $f = \sqrt{A^{2}+B^{2}}$,
$$\sigma_{f} \approx \sqrt{\left(\frac{A}{f}\right)^{2}\sigma_{A}^{2} + \left(\frac{B}{f}\right)^{2}\sigma_{B}^{2}}$$

## Import the necessary libraries

In [41]:
import yaml
import numpy as np
from statistics import mean
from numpy.random import normal
from scipy.stats import pearsonr

## List of the model names

In [42]:
lens_names = [
    "DESIJ1018-0121",
    "DESIJ1205+4110",
    "DESIJ1624+0129",
    "DESIJ0201-2739" 
]

## Uncertainty propagation

In [43]:
num_lens = len(lens_names)

data_points = []  # list dictionaries with the parameter values
center_diffs = []  # mean offsets between the light and mass centers
sigma_center_diffs = []  # uncertainty propagated standard deviation of the center offsets
size = 100  # population size
samples_Pearson = []  # calculated PCC values

for i in range(num_lens):
    lens_name = lens_names[i]

    output_path = f"../lens_systems/{lens_name}/{lens_name}_point_estimates.yml"

    try:
        with open(output_path, "r") as f:
            data = yaml.full_load(f)

    except AttributeError:
        print(f"Failed to load {lens_name}")
        continue
    

    output = {
    'center_x_light': data.get('center_x_light'),
    'center_x_mass': data.get('center_x_mass'),
    'center_y_light': data.get('center_y_light'),
    'center_y_mass': data.get('center_y_mass'),
    'Sigma_10': data.get('Sigma_10'),
    'Sigma_10_flux_selected': data.get('Sigma_10_flux_selected'),
    'Sigma_20': data.get('Sigma_20'),
    'Sigma_20_flux_selected': data.get('Sigma_20_flux_selected')
    }

    data_points.append(output)


    x_diff = abs(data_points[i]['center_x_light'][0] - data_points[i]['center_x_mass'][0])
    y_diff = abs(data_points[i]['center_y_light'][0] - data_points[i]['center_y_mass'][0])
    center_diff = (x_diff**2 + y_diff**2)**0.5
    center_diffs.append(center_diff)

    sigma_x_light = mean([data_points[i]['center_x_light'][1], data_points[i]['center_x_light'][2]])
    sigma_x_mass = mean([data_points[i]['center_x_mass'][1], data_points[i]['center_x_mass'][2]])
    sigma_y_light = mean([data_points[i]['center_y_light'][1], data_points[i]['center_y_light'][2]])
    sigma_y_mass = mean([data_points[i]['center_y_mass'][1], data_points[i]['center_y_mass'][2]])

    sigma_x_diff = (sigma_x_light**2 + sigma_x_mass**2)**0.5
    sigma_y_diff = (sigma_y_light**2 + sigma_y_mass**2)**0.5

    sigma_center_diff = (((x_diff / center_diff)**2)*sigma_x_diff**2 + ((y_diff / center_diff)**2)*sigma_y_diff**2)**0.5
    sigma_center_diffs.append(sigma_center_diff)

## Sampling and Pearson correlation coefficient (PCC) calculation

In [44]:

for i in range(size):
    samples_delta_center = []
    samples_Sigma_10 = []
    for j in range(num_lens):
        samples_delta_center.append(normal(center_diffs[j], sigma_center_diffs[j]))
        samples_Sigma_10.append(data_points[j]['Sigma_10'])
    
    samples_Pearson.append(pearsonr(samples_delta_center, samples_Sigma_10))


# print(data_points)
# print(samples_Pearson)

samples_Pearson_arr = np.array([k.statistic for k in samples_Pearson])
Pearson_median = np.median(samples_Pearson_arr)
Pearson_lower = np.percentile(samples_Pearson_arr, 16.0)
Pearson_upper = np.percentile(samples_Pearson_arr, 84.0)

print(f"Pearson Correlation Coeffient:\n\
median:{Pearson_median}\n\
upper uncertainty:{Pearson_upper - Pearson_median}\n\
lower uncertainty:{Pearson_median - Pearson_lower}")

Pearson Correlation Coeffient:
median:-0.7429244339502291
upper uncertainty:0.12306411491740654
lower uncertainty:0.0964965385299903
